# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from nltk import word_tokenize, sent_tokenize
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
nltk.download('wordnet') 
nltk.download('punkt')
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster.db')
with engine.connect() as conn, conn.begin():  
    df = pd.read_sql_table("CleanData", conn) 
X = df['message']
Y = df[df.columns.difference(['id', 'message', 'original', 'genre'])]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text = text.lower().strip()
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    cleanedtokens = [WordNetLemmatizer().lemmatize(w) for w in words]
    return cleanedtokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline =  Pipeline([
        ('features',CountVectorizer(tokenizer=tokenize)
        ),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        s...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_pred = pipeline.predict(X_test)
y_pred_np = np.array(y_pred)
y_test_np = y_test.values
for i in range(y_pred_np.shape[1]):
    print(classification_report(y_test_np[:,i], y_pred_np[:,i]))

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6475
          1       0.00      0.00      0.00        79

avg / total       0.98      0.99      0.98      6554

             precision    recall  f1-score   support

          0       0.75      0.85      0.80      3819
          1       0.74      0.59      0.66      2735

avg / total       0.75      0.75      0.74      6554

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      6227
          1       0.65      0.17      0.27       327

avg / total       0.94      0.95      0.94      6554

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6554

avg / total       1.00      1.00      1.00      6554

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      6469
          1       0.62      0.12      0.20        85

avg / total       0.98      0.99

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'features', 'clf', 'features__analyzer', 'features__binary', 'features__decode_error', 'features__dtype', 'features__encoding', 'features__input', 'features__lowercase', 'features__max_df', 'features__max_features', 'features__min_df', 'features__ngram_range', 'features__preprocessor', 'features__stop_words', 'features__strip_accents', 'features__token_pattern', 'features__tokenizer', 'features__vocabulary', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__warm_start', '

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
        'features__ngram_range': ((1, 1), (1, 2)),
        'clf__estimator__n_estimators': [10, 20, 30],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }
cv = GridSearchCV(pipeline,  param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)
y_pred_cv = cv.predict(X_test)
y_pred_cv_np = np.array(y_pred_cv)
for i in range(y_pred_np.shape[1]):
    print(classification_report(y_test_cv_np[:,i], y_pred_np[:,i]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_new =  Pipeline([
        ('features',CountVectorizer(tokenizer=tokenize)
        ),
        ('tfidf', TfidfTransformer())
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.